In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
from misc_functions import calculate_RTs


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np
import copy

from misc_functions import calculate_RTs, save_sig_chans, save_channels_to_file
import pickle

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### actually make the baselines so we can compare them later

In [18]:
def get_baseline(inst: mne.io.BaseRaw, base_times: tuple[float, float]):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")
    del inst
    return HG_base


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5), base_times=(-0.5, 0), LAB_root=None, channels=None, full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.
    
    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task. 
    High gamma (HG) will be computed, and statistics will be calculated and plotted. 
    The results will be saved to output files.
    """
    pass

sub='D0065'
task='GlobalLocal'
output_name="response_experimentStartBase1secTo101Sec"
event="Response"
times=(-1, 1.5)
base_times=(1, 301)
LAB_root=None
channels=None
full_trial_base=False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

try:
    good = crop_empty_data(filt)
except ValueError as ve:
    print(f"Error encountered during cropping: {ve}")

print(f"good channels before dropping bads: {len(good.ch_names)}")
print(f"filt channels before dropping bads: {len(filt.ch_names)}")

good.info['bads'] = channel_outlier_marker(good, 3, 2)
print("Bad channels in 'good':", good.info['bads'])

filt.drop_channels(good.info['bads']) #this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
good.drop_channels(good.info['bads'])

print("Bad channels in 'good' after dropping once:", good.info['bads'])

print(f"good channels after dropping bads: {len(good.ch_names)}")
print(f"filt channels after dropping bads: {len(filt.ch_names)}")


HG_base = get_baseline(filt, base_times)
good.load_data()

# Create a baseline EpochsTFR using the stimulus event
base_times=(-0.5,0)
adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base_stimulus = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base_stimulus, "0.5s")


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (221) does not match the number of channels in the raw data file (220). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 217 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (221) does not match the number of channels in the raw data file (220). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0065\ieeg\sub-D0065_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

good channels before dropping bads: 220
filt channels before dropping bads: 220
outlier round 1 channels: ['RPIF3']
outlier round 1 channels: ['RPIF3', 'RAIO9']
outlier round 2 channels: ['RPIF3', 'RAIO9', 'RAMT15']
outlier round 2 channels: ['RPIF3', 'RAIO9', 'RAMT15', 'RMMT8']
Bad channels in 'good': ['RPIF3', 'RAIO9', 'RAMT15', 'RMMT8']
Bad channels in 'good' after dropping once: []
good channels after dropping bads: 216
filt channels after dropping bads: 216
Reading 0 ... 6350847  =      0.000 ...  3101.000 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/n75', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s2

100%|██████████| 1/1 [03:21<00:00, 201.62s/it]


Reading 0 ... 5705315  =      0.000 ...  2785.798 secs...
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/n75', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/n75', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75', 'experimentStart']
Not setting metadata
448 matching events found
No baseline correction applied
0 projection 

100%|██████████| 448/448 [05:30<00:00,  1.36it/s]


Number of events,448
Events,Stimulus/c25/n25: 1Stimulus/c25/n75: 1Stimulus/c25/r25: 62Stimulus/c25/r75: 22Stimulus/c25/s25: 21Stimulus/c25/s75: 61Stimulus/c75/r25: 19Stimulus/c75/r75: 2Stimulus/c75/s25: 9Stimulus/c75/s75: 26Stimulus/i25/r25: 22Stimulus/i25/r75: 6Stimulus/i25/s25: 6Stimulus/i25/s75: 22Stimulus/i75/n25: 1Stimulus/i75/n75: 1Stimulus/i75/r25: 65Stimulus/i75/r75: 26Stimulus/i75/s25: 18Stimulus/i75/s75: 57
Time range,-0.500 – 0.000 s
Baseline,off


### average across trials and timepoints but not channels, plot avg mean and stdev for each channel

In [19]:
data = HG_base.get_data()

mean = np.mean(data, axis=(0,2))
stdev = np.std(data,axis=(0,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base.ch_names

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(channel_names, mean, yerr=stdev, align='center', alpha=0.75, ecolor='black', capsize=5)

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.title('Mean and Standard Deviation of Channels')
plt.xticks(rotation=90)  # Rotate the channel names for better readability

# Display the plot
plt.tight_layout()
plt.show()


Mean:  [           nan            nan            nan            nan
            nan            nan 9.35136258e-06            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan 6.13801923e-06            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
 5.01278029e-06  

KeyboardInterrupt: 

In [20]:
data = HG_base_stimulus.get_data()

mean = np.mean(data, axis=(0,2))
stdev = np.std(data,axis=(0,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(channel_names, mean, yerr=stdev, align='center', alpha=0.75, ecolor='black', capsize=5)

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.title('Mean and Standard Deviation of Channels')
plt.xticks(rotation=90)  # Rotate the channel names for better readability

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [2.35596596e-05            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
 7.60397970e-06 7.19475317e-06 6.55293008e-06 6.04449749e-06
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan  

KeyboardInterrupt: 

### average across trials, channels, and time points

In [21]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of experiment start baseline: ", mean)
print("Standard Deviation of experiment start baseline: ", stdev)

Mean of experiment start baseline:  7.11109295788856e-06
Standard Deviation of experiment start baseline:  2.467975522575185e-06


In [22]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of fixation cross baseline: ", mean)
print("Standard Deviation of fixation cross baseline: ", stdev)

Mean of fixation cross baseline:  6.0587577502376924e-06
Standard Deviation of fixation cross baseline:  4.567779012040867e-06


### average across trials and channels but not timepoints

In [23]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base.times, mean, label='Mean')
plt.fill_between(HG_base.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [6.21817376e-06 6.23033093e-06 6.24244690e-06 ... 7.53210802e-06
 7.54389375e-06 7.55552610e-06]
Standard Deviation:  [1.16503499e-06 1.17829252e-06 1.19179487e-06 ... 2.64761902e-06
 2.64209893e-06 2.63672973e-06]


KeyboardInterrupt: 

In [24]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base_stimulus.times, mean, label='Mean')
plt.fill_between(HG_base_stimulus.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [5.96941022e-06 5.96859328e-06 5.96779262e-06 ... 6.24856718e-06
 6.24944321e-06 6.25032414e-06]
Standard Deviation:  [4.04477745e-06 4.03667350e-06 4.02874892e-06 ... 4.98159151e-06
 4.97979517e-06 4.97793714e-06]


KeyboardInterrupt: 